In [44]:
from selenium import webdriver


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

browser = webdriver.Edge()
browser.get('https://www.google.com/')


In [72]:
import requests
import json
#墾丁凱薩_按最新排序_顧客評論
link = "https://www.booking.com/reviewlist.zh-tw.html?aid=309654&label=hotels-chinese-zh-xt-not-tw-3E%2AoDaZgyjzN82axX_iA0wS168630596047%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-320553767261%3Alp9051484%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YcsZ-Id2vkzIfTmYhvC5HOg&sid=612b1d9bc12dd29048d367a325a1adf8&srpvid=6379249454980051&;cc1=tw&pagename=caesar-park-kenting&r_lang=&review_topic_category_id=&type=total&score=&sort=f_recent_desc&time_of_year=&dist=1&offset=&rows=10&rurl=&text=&translate=&_=1687267902319"
browser.get(link)
time.sleep(2)

In [73]:

saved_review = []


def get_current_page():
    element = browser.find_element(By.CSS_SELECTOR, ".bui-pagination__item.bui-pagination__item--active")
    return element.find_element(By.CSS_SELECTOR, "span[aria-hidden='true']").text
    


def scrape_review(current_page):
    global saved_review
    blocks = browser.find_elements(By.CLASS_NAME, "c-review-block__right")
    

    # 逐一處理每個評論區塊
    for block in blocks:
        review_dict = {
            'title': None,
            'positive_review': None,
            'negative_review': None
        }

        # 找到標題元素
        try:
            title_element = block.find_element(By.CLASS_NAME, "c-review-block__title")
            review_dict['title'] = title_element.text
        except NoSuchElementException:
                pass

        # 找到評論元素
        review_blocks = block.find_elements(By.CLASS_NAME, "c-review")
        for review_block in review_blocks:
            emotion_reviews = review_block.find_elements(By.CLASS_NAME, "c-review__row")

            for emotion_review in emotion_reviews:
                try:
                    review_prefix = emotion_review.find_element(By.CLASS_NAME, "c-review__prefix").text
                    review_body = emotion_review.find_element(By.CLASS_NAME, "c-review__body").text

                    if "喜歡" in review_prefix:
                        if review_dict['positive_review'] is None:
                            review_dict['positive_review'] = review_body
                    if "不喜歡" in review_prefix:
                        if review_dict['negative_review'] is None:
                            review_dict['negative_review'] = review_body
                        
                except NoSuchElementException:
                    pass
                
    

        saved_review.append(review_dict)
        

    print(f"Page{current_page}'s review is saved")

    
def main():
    for __ in range(231):
        current_page = get_current_page()

        scrape_review(current_page)

        next_page_button = browser.find_element(By.CLASS_NAME,'pagenext')
        next_page_button.click()
        time.sleep(2)

main()




Page1's review is saved
Page2's review is saved
Page3's review is saved
Page4's review is saved
Page5's review is saved
Page6's review is saved
Page7's review is saved
Page8's review is saved
Page9's review is saved
Page10's review is saved
Page11's review is saved
Page12's review is saved
Page13's review is saved
Page14's review is saved
Page15's review is saved
Page16's review is saved
Page17's review is saved
Page18's review is saved
Page19's review is saved
Page20's review is saved
Page21's review is saved
Page22's review is saved
Page23's review is saved
Page24's review is saved
Page25's review is saved
Page26's review is saved
Page27's review is saved
Page28's review is saved
Page29's review is saved
Page30's review is saved
Page31's review is saved
Page32's review is saved
Page33's review is saved
Page34's review is saved
Page35's review is saved
Page36's review is saved
Page37's review is saved
Page38's review is saved
Page39's review is saved
Page40's review is saved
Page41's 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pagenext"}
  (Session info: MicrosoftEdge=114.0.1823.55)
Stacktrace:
0   msedgedriver                        0x0000000105120f20 msedgedriver + 4394784
1   msedgedriver                        0x0000000105119234 msedgedriver + 4362804
2   msedgedriver                        0x0000000104d2e56c msedgedriver + 255340
3   msedgedriver                        0x0000000104d65938 msedgedriver + 481592
4   msedgedriver                        0x0000000104d97ea4 msedgedriver + 687780
5   msedgedriver                        0x0000000104d5bfcc msedgedriver + 442316
6   msedgedriver                        0x0000000104d5d14c msedgedriver + 446796
7   msedgedriver                        0x00000001050e88cc msedgedriver + 4163788
8   msedgedriver                        0x00000001050ec284 msedgedriver + 4178564
9   msedgedriver                        0x00000001050ebe54 msedgedriver + 4177492
10

In [80]:
import csv

filename = "/Users/chris/Desktop/Net_Scrapping/Selenium_driver/output/bkcom_review_raw.csv"

with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    # 定義 CSV 欄位
    fieldnames = ['title', 'positive_review', 'negative_review']

    # 建立 CSV 寫入器
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入標題行
    writer.writeheader()

    # 寫入資料行
    for review in saved_review:
        writer.writerow(review)

In [77]:
for review in saved_review:
    print(review)


{'title': '傑出', 'positive_review': None, 'negative_review': None}
{'title': '令人愉悅', 'positive_review': None, 'negative_review': None}
{'title': '位置雖佳.但價格不便宜.飯店不少小細節不貼心.沒有高級飯店的水準.應該不會再去了.', 'positive_review': '位置佳.早餐多樣豐富好吃.服務人員態度親切積極.', 'negative_review': '網路差.很差.非常差....戶外泡澡池材質很粗造.踩踏下去很不舒服.若是跪著.膝蓋很痛..'}
{'title': '傑出', 'positive_review': None, 'negative_review': None}
{'title': '傑出', 'positive_review': None, 'negative_review': None}
{'title': '好極了', 'positive_review': None, 'negative_review': None}
{'title': '傑出', 'positive_review': None, 'negative_review': None}
{'title': '非常好', 'positive_review': '房務員來加裝小朋友護欄，很親切、用心。 去沙灘可以登記使用沙灘傘跟躺椅，一房一組。', 'negative_review': '早餐選第一時段，興許是入住人數眾多，桌數不夠/整理不及，導致住客在外排隊等待入席，但不巧用餐時段快結束，吃得很匆忙。'}
{'title': '好極了', 'positive_review': '房間乾淨整潔，自然光線充足明亮，後門直通飯店泳池，非常方便，室外的浴池夠大，給小朋友玩水也很適合，房間的戶外區有僞芭厘島風，停車用餐皆便利，過馬路就是海灘。', 'negative_review': None}
{'title': '浴室拉門有點問題,無法拉動門,有反應給房務,但尚未維修', 'positive_review': None, 'negative_review': None}
{'title': '傑出', 'positive_review': 